In [1]:
import os
import glob
import json
import tsfresh
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# Data Directory structure

MyShake_Trainig_Data  
|-- EQ  
----|-- shake_table (193 files)  
----|-- simulated (993 files)  
|-- Human (26344 files)  

In [22]:
table_files = glob.glob('./MyShake_Training_Data/EQ/shake_table/*')
simulated_files = glob.glob('./MyShake_Training_Data/EQ/simulated/*')
human_files = glob.glob('./MyShake_Training_Data/Human/*')

# Creating DataFrames for one data file

In [2]:
def all_data(file):
    'Get dictionary from JSON file'
    with open(file, encoding='utf-8') as f:
        data=[json.loads(line) for line in f]
        data=data[0]
    return data

In [24]:
test_file = 'MyShake_Training_Data/Human/013306004148017_1415046600.json'
test_data = all_data(test_file)

In [25]:
test_data['header']

{'sampling_rate': 25.0,
 'starttime': 1415046600,
 'station': '013306004148017',
 'stla': 12345,
 'stlo': 12345,
 'triggertime': 1415046662.432}

In [6]:
test_df_1 = pd.DataFrame(test_data['data'])

In [26]:
test_df_1.describe()

,x,y,z,t,delta_t
count,7403.000000,7403.000000,7403.000000,7403.000000,7403.000000
mean,-0.024189,0.031678,-1.001861,148.039859,148.039859
std,0.003427,0.003656,0.003760,85.488173,85.488173
min,-0.057600,0.007281,-1.014853,0.000000,0.000000
25%,-0.027000,0.029400,-1.004829,74.019929,74.019929
50%,-0.023780,0.031000,-1.002000,148.039859,148.039859
75%,-0.022610,0.034415,-0.998976,222.059788,222.059788
max,0.011400,0.064488,-0.983454,296.079718,296.079718


In [8]:
def get_time(data):
    t0 = data['header']['starttime']
    npoints = min(len(data['data']['x']), len(data['data']['y']), len(data['data']['z'])) 
    sampling_rate = data['header']['sampling_rate']
    t1 = t0 + npoints / sampling_rate
    # form the timestamp
    t = np.arange(t0, t1, 1/sampling_rate)
    return t0, t[:npoints]

def get_df(file):
    data = all_data(file)
    df = pd.DataFrame(data['data'])
    t0, t = get_time(data)
    df['t'] = t
    df['delta_t'] = t-t0
    df['filename'] = file[len(dir_name):]
    return df

# Creating DataFrame for entire directory

## Human

In [ ]:
# length of file

test_data = all_data(test_file)
test_df_1 = pd.DataFrame(test_data['data'])

t0 = test_data['header']['starttime']
npoints = max(len(test_data['data']['x']), len(test_data['data']['y']), len(test_data['data']['z'])) 
sampling_rate = test_data['header']['sampling_rate']
t1 = t0 + npoints / sampling_rate
# form the timestamp
t = np.arange(t0, t1, 1/sampling_rate)

test_df_1['delta_t'] = t 
test_df_1['delta_t'] = t - t0

test_df_1['filename'] = test_file[28:]

In [ ]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

def extract_features(parent_dir,sub_dirs,file_ext="*.wav",bands = 60, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    labels = []
    for l, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            sound_clip,s = librosa.load(fn)
            label = fn.split('/')[2].split('-')[1]
            for (start,end) in windows(sound_clip,window_size):
                if(len(sound_clip[start:end]) == window_size):
                    signal = sound_clip[start:end]
                    melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                    logspec = librosa.logamplitude(melspec)
                    logspec = logspec.T.flatten()[:, np.newaxis].T
                    log_specgrams.append(logspec)
                    labels.append(label)
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
    
    return np.array(features), np.array(labels,dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [ ]:
features,labels = extract_features(parent_dir,sub_dirs)
labels = one_hot_encode(labels)

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(1.0, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x,W,strides=[1,2,2,1], padding='SAME')

def apply_convolution(x,kernel_size,num_channels,depth):
    weights = weight_variable([kernel_size, kernel_size, num_channels, depth])
    biases = bias_variable([depth])
    return tf.nn.relu(tf.add(conv2d(x, weights),biases))

def apply_max_pool(x,kernel_size,stride_size):
    return tf.nn.max_pool(x, ksize=[1, kernel_size, kernel_size, 1], 
                          strides=[1, stride_size, stride_size, 1], padding='SAME')

In [ ]:
rnd_indices = np.random.rand(len(labels)) < 0.70

train_x = features[rnd_indices]
train_y = labels[rnd_indices]
test_x = features[~rnd_indices]
test_y = labels[~rnd_indices]

frames = 41
bands = 60

feature_size = 2460 #60x41
num_labels = 10
num_channels = 2

batch_size = 50
kernel_size = 30
depth = 20
num_hidden = 200

learning_rate = 0.01
training_iterations = 2000

In [ ]:
X = tf.placeholder(tf.float32, shape=[None,bands,frames,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])

cov = apply_convolution(X,kernel_size,num_channels,depth)

shape = cov.get_shape().as_list()
cov_flat = tf.reshape(cov, [-1, shape[1] * shape[2] * shape[3]])

f_weights = weight_variable([shape[1] * shape[2] * depth, num_hidden])
f_biases = bias_variable([num_hidden])
f = tf.nn.sigmoid(tf.add(tf.matmul(cov_flat, f_weights),f_biases))

out_weights = weight_variable([num_hidden, num_labels])
out_biases = bias_variable([num_labels])
y_ = tf.nn.softmax(tf.matmul(f, out_weights) + out_biases)

In [ ]:
cross_entropy = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
cost_history = np.empty(shape=[1],dtype=float)
with tf.Session() as session:
    tf.initialize_all_variables().run()

    for itr in range(training_iterations):    
        offset = (itr * batch_size) % (train_y.shape[0] - batch_size)
        batch_x = train_x[offset:(offset + batch_size), :, :, :]
        batch_y = train_y[offset:(offset + batch_size), :]
        
        _, c = session.run([optimizer, cross_entropy],feed_dict={X: batch_x, Y : batch_y})
        cost_history = np.append(cost_history,c)
    
    print('Test accuracy: ',round(session.run(accuracy, feed_dict={X: test_x, Y: test_y}) , 3))
    fig = plt.figure(figsize=(15,10))
    plt.plot(cost_history)
    plt.axis([0,training_epochs,0,np.max(cost_history)])
    plt.show()